In [3]:
from library.objective_function import *
from library.optimiser import *
from library.post_analysis import *
from library.experiments import *
%matplotlib inline
SEED = 1231

In [4]:
def do_multiple_exps(obj, opt, zoneParas):
    np.random.seed(SEED)
    
    # init multiple experiments
    mexps = multiple_experiment()

    # init one experiment
    exp = single_experiment()
    
    # One experiment: setup objective function and optimiser
    exp.set_objective(obj)
    exp.set_optimizer(opt)

    # Multiple experiment: setup one expreiment
    mexps.set_single_exp(exp)
    # Multiple experiment: setup sample zone    
    mexps.set_sample_zone(zoneParas)
    
    data = mexps.do()
    
    return data

In [7]:
# 1 optimiser
inner_opt = line_search()
innerOptParas = {
    'x0': None,
    'alpha': 1,
    'beta': 0.1, 
    'max_iter': 100,
    'tol': 1e-2,              
    'verbose': False,
    'record': False
}
inner_opt.set_parameters(innerOptParas)

opt = cma_es()
optParas ={'x0': np.zeros((2,)),
           'std': np.ones((2,)) * 3, 
           'tol': 1e-3, 
           'adjust_func': inner_opt, 
           'record': False, 
           'verbose': False}
opt.set_parameters(optParas)

# 2 objective function
obj = eggholder()

# 3 setup sample zone of multiple experiments
zoneParas = {'origin': np.array([-550,-550]),
             'edge': np.array([1100, 1100]),
             'step': 550,
             'size': 10,
             'sym': False}

data = do_multiple_exps(obj, opt, zoneParas)

/Users/huajian/Non-convex/library/objective_function.py:176: RuntimeWarning:

divide by zero encountered in double_scalars

/Users/huajian/Non-convex/library/objective_function.py:176: RuntimeWarning:

invalid value encountered in double_scalars



cost: 330.7446182005433, prob: 0.0
complete: 2 / 4 
cost: 270.62116424046513, prob: 0.1
complete: 4 / 4 
avg probility of convergence:  0.025
avg cost:  312.83334313370943
avg evals per exp:  30259.8
total time: 29.350615978240967,  time per eval:6.0622128984331045e-05

